# Climate Tipping Point Predictor: Predicting Climate Thresholds Where CO₂ Emissions Trigger Accelerated Sea Level Rise Using Temporal Causal Modeling

Problem Statement:
    
Despite growing awareness of climate change, the nonlinear relationship between CO₂ emissions and sea level rise remains poorly understood. 
This project aims to develop a temporal machine learning model that identifies critical thresholds of CO₂ emissions beyond which sea level 
rise accelerates disproportionately. By integrating time-series data on temperature, precipitation, humidity, and wind speed, the model will 
uncover hidden causal patterns and predict when and where climate tipping points may occur.


In [2]:
import pandas as pd

In [4]:
df = pd.read_csv("C:/Users/KIIT0001/OneDrive/Desktop/MAHASWETA/EduNet/climate_change_data.csv")
df.head()

,Date,Location,Country,Temperature,CO2 Emissions,Sea Level Rise,Precipitation,Humidity,Wind Speed
0,2000-01-01 00:00:00.000000000,New Williamtown,Latvia,10.688986,403.118903,0.717506,13.835237,23.631256,18.492026
1,2000-01-01 20:09:43.258325832,North Rachel,South Africa,13.814430,396.663499,1.205715,40.974084,43.982946,34.249300
2,2000-01-02 16:19:26.516651665,West Williamland,French Guiana,27.323718,451.553155,-0.160783,42.697931,96.652600,34.124261
3,2000-01-03 12:29:09.774977497,South David,Vietnam,12.309581,422.404983,-0.475931,5.193341,47.467938,8.554563
4,2000-01-04 08:38:53.033303330,New Scottburgh,Moldova,13.210885,410.472999,1.135757,78.695280,61.789672,8.001164


Data Preprocessing

In [9]:
df.columns

Index(['Date', 'Location', 'Country', 'Temperature', 'CO2 Emissions',
       'Sea Level Rise', 'Precipitation', 'Humidity', 'Wind Speed'],
      dtype='object')

In [11]:
# Checking for null values
df.isnull().sum()

Date              0
Location          0
Country           0
Temperature       0
CO2 Emissions     0
Sea Level Rise    0
Precipitation     0
Humidity          0
Wind Speed        0
dtype: int64

In [12]:
# Convert 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

In [13]:
# Droping duplicates
df.drop_duplicates(inplace=True)

In [14]:
# Standardize column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

In [15]:
# Encoding categorical columns 
df['location'] = df['location'].astype('category')
df['country'] = df['country'].astype('category')

In [16]:
# Extract date features 
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

In [17]:
df.head()

,date,location,country,temperature,co2_emissions,sea_level_rise,precipitation,humidity,wind_speed,year,month,day
0,2000-01-01 00:00:00.000000000,New Williamtown,Latvia,10.688986,403.118903,0.717506,13.835237,23.631256,18.492026,2000,1,1
1,2000-01-01 20:09:43.258325832,North Rachel,South Africa,13.814430,396.663499,1.205715,40.974084,43.982946,34.249300,2000,1,1
2,2000-01-02 16:19:26.516651665,West Williamland,French Guiana,27.323718,451.553155,-0.160783,42.697931,96.652600,34.124261,2000,1,2
3,2000-01-03 12:29:09.774977497,South David,Vietnam,12.309581,422.404983,-0.475931,5.193341,47.467938,8.554563,2000,1,3
4,2000-01-04 08:38:53.033303330,New Scottburgh,Moldova,13.210885,410.472999,1.135757,78.695280,61.789672,8.001164,2000,1,4
